<a href="https://colab.research.google.com/github/flydragon2018/Analytics_Vidhya/blob/master/hotel_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive') #, force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip -q /content/drive/"My Drive"/hotel/Hotel_rating.zip


replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
!git clone https://github.com/Shemka/ReviewsScoring.git

fatal: destination path 'ReviewsScoring' already exists and is not an empty directory.


In [4]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [5]:
!cat  /content/ReviewsScoring/requirements.txt

In [6]:
!sed 's/en-core-web-sm==2.1.0/en-core-web-sm/' /content/ReviewsScoring/requirements.txt > /content/ReviewsScoring/requirements.txt

In [7]:
!pip install -r /content/ReviewsScoring/requirements.txt

In [8]:
#!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
#!gunzip GoogleNews-vectors-negative300.bin.gz

In [9]:
#!cp -rf /content/ReviewsScoring/* .

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
# MAIN
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter
import gc
import pickle
import joblib
from scripts.storing import save_model, load_model
from scripts.cleanup import CleanUpText

# NLP
import spacy
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
import gensim
from gensim.models import Word2Vec

# SKLEARN
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, mean_squared_error, r2_score

# TENSORFLOW
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, Flatten, Conv1D, SpatialDropout1D, MaxPooling1D, Dense, GRU, LSTM, Dropout, BatchNormalization, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import LearningRateScheduler, EarlyStopping

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
reviews = pd.read_csv('train.csv') 
#reviews = reviews[['Reviewer_Score', 'review']]
reviews.head(5)

,review,rating
0,good place stay check rainforest biobay vieque...,4
1,"great firstly did n't enjoy hong kong, 3 days ...",5
2,clean convenient hotel catedral ideally locate...,4
3,transport good class high excellent communicat...,3
4,"stay happy la quinta, used stay travelodge str...",4


In [13]:
text = CleanUpText().fit_transform(reviews['review'])

100%|██████████| 16392/16392 [00:21<00:00, 746.69it/s]


In [14]:
# Data splitting and tokenizing 
X_train, X_val, y_train, y_val = train_test_split(text, reviews['rating']/5, random_state=99, test_size=0.1)
# Vocabulary len
NUM_WORDS = 50000

tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 35461 unique tokens.


In [15]:

# Save tokenizer object
save_model(tokenizer, 'my_tokenizer.pkl')

Model was saved in my_tokenizer.pkl
Use load_model to load model from file.


In [16]:
%%time
# Sequence cropping
X_train = pad_sequences(X_train)
X_val = pad_sequences(X_val, maxlen=X_train.shape[1])
print('Shape of X train and X validation tensor:', X_train.shape, X_val.shape)

Shape of X train and X validation tensor: (14752, 1505) (1640, 1505)
CPU times: user 217 ms, sys: 16.1 ms, total: 233 ms
Wall time: 234 ms


In [17]:
%%time
MAX_LEN = X_train.shape[1]
gwv = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

CPU times: user 33.9 s, sys: 4.13 s, total: 38 s
Wall time: 1min 38s


In [18]:
%%time
# Create embedding matrix
count_null = 0 
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = gwv[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        count_null += 1
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)
print('Count of string that not in word2vec vocabulary:', count_null)
print('From %d words' % (vocabulary_size))
del(gwv); gc.collect()

Count of string that not in word2vec vocabulary: 14157
From 35462 words
CPU times: user 790 ms, sys: 54.9 ms, total: 845 ms
Wall time: 844 ms


In [19]:
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_shape=(MAX_LEN,),
                            trainable=False)

In [25]:
optimizer_adam = Adam(lr = 0.0005)

callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=10, verbose=1, restore_best_weights=True)
]

In [26]:
model = Sequential()
model.add(embedding_layer)
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(GRU(32, return_sequences=True)))
model.add(Conv1D(32, 11, activation='relu'))
model.add(MaxPooling1D(5))
model.add(SpatialDropout1D(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_absolute_error', optimizer=optimizer_adam, metrics=['mae', 'mse'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1505, 300)         10638600  
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 1505, 300)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1505, 64)          64128     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1495, 32)          22560     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 299, 32)           0         
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 299, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9568)             

In [27]:
# It's just an example of training a model. Real model have another loss
model.fit(X_train, np.asarray(y_train), batch_size=512, epochs=120, verbose=1, 
          callbacks=callbacks, validation_data=(X_val, np.asarray(y_val)))

Epoch 1/120
29/29 [==============================] - 16s 566ms/step - loss: 0.2333 - mae: 0.2333 - mse: 0.0979 - val_loss: 0.2083 - val_mae: 0.2083 - val_mse: 0.1003
Epoch 2/120
29/29 [==============================] - 16s 549ms/step - loss: 0.2088 - mae: 0.2088 - mse: 0.1041 - val_loss: 0.2084 - val_mae: 0.2084 - val_mse: 0.1007
Epoch 3/120
29/29 [==============================] - 16s 552ms/step - loss: 0.2088 - mae: 0.2088 - mse: 0.1038 - val_loss: 0.2081 - val_mae: 0.2081 - val_mse: 0.0999
Epoch 4/120
29/29 [==============================] - 16s 546ms/step - loss: 0.2079 - mae: 0.2079 - mse: 0.0998 - val_loss: 0.1985 - val_mae: 0.1985 - val_mse: 0.0794
Epoch 5/120
29/29 [==============================] - 16s 546ms/step - loss: 0.1825 - mae: 0.1825 - mse: 0.0585 - val_loss: 0.1526 - val_mae: 0.1526 - val_mse: 0.0406
Epoch 6/120
29/29 [==============================] - 16s 543ms/step - loss: 0.1456 - mae: 0.1456 - mse: 0.0392 - val_loss: 0.1336 - val_mae: 0.1336 - val_mse: 0.0349
Epoc

In [28]:
test=pd.read_csv('test.csv')
test.head()

,id,review
0,0,ehhh better punta cana twice compared hotel st...
1,1,"4 n't think, decided book atenea night stay de..."
2,2,awesome time just returned vacation fantastic ...
3,3,"grand oasis wonderful second time, group 20 fr..."
4,4,not bad stay stayed hotel family attending jav...


In [29]:
import tensorflow as tf
from scripts.review_scoring import ReviewScorer

In [33]:
!cp *.pkl /content/models/

In [35]:
scorer = ReviewScorer('models/model.h5', 'models/my_tokenizer.pkl')

Model was loaded from models/my_tokenizer.pkl


In [37]:
p=scorer.predict(['It\'s the best hotel I ever see'])

In [38]:
p[0]

0.8338482

In [ ]:
test['rating']=0
for i in test.index:
  text=test['review'][i]
  score=scorer.predict([text])
  s=int(round(score[0]*5))
  test['rating'][i]=s
  print(i,s)
  
  #if i==10:
  #  break

test.to_csv("rating_sub.csv",columns=['id','rating'],header=None,index=None)

0 3
1 3
2 4
3 4
4 4
5 5
6 4
7 4
8 5
9 2
10 4
11 4
12 4
13 4
14 3
15 4
16 4
17 3
18 4
19 4
20 4
21 3
22 4
23 4
24 4
25 4
26 4
27 4
28 5
29 4
30 4
31 3
32 4
33 4
34 4
35 3
36 4
37 3
38 4
39 3
40 4
41 4
42 5
43 4
44 4
45 4
46 5
47 4
48 5
49 4
50 5
51 4
52 4
53 3
54 4
55 4
56 3
57 4
58 4
59 3
60 4
61 3
62 3
63 4
64 5
65 4
66 5
67 4
68 4
69 4
70 5
71 5
72 3
73 3
74 4
75 4
76 4
77 4
78 4
79 4
80 5
81 4
82 3
83 4
84 4
85 4
86 5
87 3
88 4
89 4
90 5
91 4
92 5
93 4
94 5
95 3
96 5
97 4
98 4
99 4
100 4
101 5
102 4
103 4
104 5
105 4
106 4
107 5
108 5
109 3
110 4
111 4
112 4
113 4
114 4
115 3
116 5
117 4
118 5
119 3
120 3
121 3
122 4
123 3
124 5
125 4
126 5
127 4
128 4
129 4
130 3
131 4
132 3
133 4
134 4
135 3
136 4
137 4
138 4
139 3
140 4
141 5
142 3
143 4
144 4
145 4
146 3
147 5
148 4
149 4
150 5
151 5
152 4
153 4
154 3
155 4
156 3
157 4
158 5
159 5
160 3
161 4
162 4
163 5
164 4
165 4
166 4
167 4
168 5
169 3
170 5
171 4
172 4
173 5
174 3
175 4
176 4
177 4
178 3
179 4
180 4
181 3
182 3
183 3
184 3
